### Import Function and Libraries

In [25]:
import pandas as pd 
import numpy as np 
import yfinance as yf
from yahoofinancials import YahooFinancials
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
from Data_Retrieval import DataRetrieval
from CC_test import CCstudy
import itertools
from pandas_datareader import data as pdr
from yahooquery import Ticker

In [27]:
from Spread import Spread

In [29]:
from time import time
from pathlib import Path
from tqdm import tqdm 

import numpy as np
from numpy.linalg import LinAlgError
import pandas as pd

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.tree import  DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV

from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.api import VAR

import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
import warnings
warnings.filterwarnings('ignore')

### Industry and Sector dictionary

In [34]:
def classification(tickers):
    tickers = Ticker(tickers, asynchronous=True)
    
    datasi = tickers.get_modules("summaryProfile quoteType")
    dfsi = pd.DataFrame.from_dict(datasi).T
    dataframes = [pd.json_normalize([x for x in dfsi[module] if isinstance(x, dict)]) for
    module in ['summaryProfile', 'quoteType']]
    
    dfsi = pd.concat(dataframes, axis=1)
    
    dfsi = pd.DataFrame(dfsi.set_index('symbol'))
    dfsi=dfsi[['industry', 'sector']].reset_index()
    dfsi['symbol'].str.upper()
    return dfsi


### Import TW exchange counters

Our main focus in on Taiwan exchange counters hence we will import taiwan exchange ticker code and retrieve the adjusted closing price from yahoo finance API.

In [36]:
counter_code = pd.read_csv('taiwan_stocks.csv')

In [8]:
tickers=['^TWII','2330.TW']

In [38]:
tickers = list(counter_code.iloc[:,2])

In [40]:
tickers = list(counter_code.iloc[:,2].apply(lambda x: str(x)+".TW"))

In [42]:
classlist = classification(tickers)

In [43]:
classlist

,symbol,industry,sector
0,1303.TW,Specialty Chemicals,Basic Materials
1,2881.TW,Insurance - Life,Financial Services
2,2002.TW,Steel,Basic Materials
3,2303.TW,Semiconductors,Technology
4,2880.TW,Banks - Regional,Financial Services
...,...,...,...
188,1503.TW,Electrical Equipment & Parts,Industrials
189,5876.TW,Banks - Regional,Financial Services
190,6139.TW,Engineering & Construction,Industrials
191,6526.TW,Semiconductors,Technology


In [44]:
#tickers = ['2330.tw','0052.TW']
start_date = "2020-01-01"
end_date = "2023-12-30"

if tickers is not None:
    merged_df = DataRetrieval.retrieve_and_merge_data(tickers, start_date, end_date)
else:
    print("Error: tickers is None")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [48]:
merged_df = merged_df.dropna(axis=1, how='any')

### Correlation and Cointegration Study 

In [ ]:
cc_study = CCstudy(merged_df)
table = cc_study.main()


Cointegration Test Results:


In [ ]:
table.sort_values(by=['Cointegration result','correlation'],ascending =[True,False])

In [ ]:
table =table[(table['Cointegration result'] < 0.05) & (table['correlation'] > 0.5)].sort_values(by=['correlation','Cointegration result'],ascending =[False,True])

In [ ]:
def categorize(classlist,column,counter):
    if column == 'industry':
        a=pd.DataFrame(dict(classlist.iloc[:,0:2]))
        return table[counter].map(a.set_index('symbol')[column])
    elif column == 'sector':
        b= pd.DataFrame(dict(classlist.iloc[:,[0,2]]))
        return table[counter].map(b.set_index('symbol')[column])
    else:
        return 'Error'

In [ ]:
classlist

In [ ]:
table['ticker1_industry'] = categorize(classlist,'sector','ticker1')
table['ticker2_industry'] = categorize(classlist,'sector','ticker2')


In [ ]:
table

### Selection of pair trading

In order to remove any industry specific systematic risk, the first criteria should be that the pair of stocks should belong to the same industry

In [ ]:
table =table[(table['Cointegration result'] < 0.05) & (table['ticker1_industry'] == table['ticker2_industry'])].sort_values(by=['correlation','Cointegration result'],ascending =[False,True])

In [ ]:
table[table['ticker1_industry'] == 'Technology']

In [ ]:
Pair = ['^TWII','2330.TW']	

In [ ]:
spread_calculator = Spread(merged_df)
spread_calculator.calculate_spread_and_volatility(Pair)